In [2]:
'''
Notebook for FIGS6 - Adjusting the punisher's switching timescale
'''
# By Kirill Sechkar

# PACKAGE IMPORTS ------------------------------------------------------------------------------------------------------
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Heun, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
import pickle
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, transform as bktransform
from math import pi
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
from contourpy import contour_generator as cgen
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 
bkplot.output_backend = 'svg'

# OWN CODE IMPORTS -----------------------------------------------------------------------------------------------------
import synthetic_circuits as circuits
from cell_model import *
from get_steady_state import *
from Fig2.design_guidance_tools import *

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [3]:
 # INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.twotoggles_punisher_initialise,
    circuits.twotoggles_punisher_ode,
    circuits.twotoggles_punisher_F_calc,
    par, init_conds)  # load the circuit

In [4]:
# SPECIFY THE CIRCUIT'S DEFAULT PARAMETERS

# TOGGLE SWITCHES
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['c_tog' + str(togswitchnum) + str(toggenenum)] = 1  # copy no. (nM)
        par['a_tog' + str(togswitchnum) + str(toggenenum)] = 1e5/2  # promoter strength (unitless)

        # transcription regulation function
        reg_func_string = 'dna(tog' + str(togswitchnum) + str(toggenenum) + '):p_tog' + str(togswitchnum) + str(
            (toggenenum - 2) % 2 + 1)  # dna(rep1):p_rep3, dna(rep2):p_rep1 or dna(rep3):p_rep2
        par['K_' + reg_func_string] = 2500  # half-saturation constant
        par['eta_' + reg_func_string] = 2  # Hill coefficient
        par['baseline_tog' + str(togswitchnum) + str(
            toggenenum)] = 0.025  # baseline transcription activation due to leakiness
        par['p_tog' + str(togswitchnum) + str(
            toggenenum) + '_ac_frac'] = 1  # active fraction of protein (i.e. share of molecules not bound by the inducer)
    # break symmetry for each of the toggle switches
    init_conds['m_tog' + str(togswitchnum) + '1'] = 4000

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0#0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['baseline_switch_alt'] = 0
par['p_switch_ac_frac'] = 0.88  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

In [5]:
# get the cellular variables in steady state without burden
e, F_r, h, xis, Chis = values_for_analytical(par, ode_with_circuit, init_conds,
                                          circuit_genes, circuit_miscs,
                                          circuit_name2pos,
                                          circuit_F_calc)
# record the cellular variables
cellvars = {'e': e, 'F_r': F_r, # translation elongation rate and ribosome trnscription regulation
            'h': h, # intacellular chlorampenicol concentration
            # burden values
            'xi_a': xis['a'], 'xi_r': xis['r'], 'xi_other_genes': xis['other'], 'xi_cat': xis['cat'],
            'xi_switch_max': xis['switch (max)'], 'xi_int_max': xis['int (max)'], 'xi_prot': xis['prot'],
            # protein degradation correction factors for the switch protein and the integrase
            'chi_switch': Chis['switch'], 'chi_int': Chis['int']}

In [6]:
# SIMULATE HOW THE PUNISHER GETS SWITCHED ON BY A TRANSIENT FLUCTUATION IN BURDEN

# SET DETERMINISTIC SIMULATION PARAMETERS
# set up the diffrax simulator
savetimestep = 0.05  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

# SIMULATION TIME FRAMES
# getting the initial state
tf_prepun = (0,25+savetimestep/2) # time frame for simulation before punisher comes online - to avoid premature tiggering
tf_pun = (25,50+savetimestep/2) # time frame for simulation after punisher comes online
# flipping both toggles with an inducer pulse
tf_pulse = (50,50.75+savetimestep/2) # time frame for inducer pulse that flips the toggle
tf_afterpulse = (50.75,85+savetimestep/2) # time frame for simulation after the pulse

# flipping a toggle: inducer pulse parameters
pulse_ptog11_acfrac = 0.0 # active fraction of p_tog11 during the inducer pulse
pulse_ptog21_acfrac = 0.0 # active fraction of p_tog21 during the inducer pulse

# SIMULATE
par_flip = par.copy()

# initial simulation to get the steady state without gene expression loss
ac_frac = par_flip['p_switch_ac_frac']
par_flip['p_switch_ac_frac'] = 0.0  # first with punisher inactive to avoid it being triggered
sol = ode_sim(par_flip,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_flip, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf_prepun, jnp.arange(tf_prepun[0], tf_prepun[1]+savetimestep/2, savetimestep),  # time frame and time axis for saving the system's state
              rtol, atol)  # relative and
ts = np.array(sol.ts)
xs = np.array(sol.ys)
par_flip['p_switch_ac_frac'] = ac_frac
sol_punisher_active = ode_sim(par_flip,  # dictionary with model parameters
                              ode_with_circuit,  # ODE function for the cell with synthetic circuit
                              sol.ys[-1, :],
                              # initial condition VECTOR
                              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                              cellmodel_auxil.synth_gene_params_for_jax(par_flip, circuit_genes),
                              # synthetic gene parameters for calculating k values
                              tf_pun, jnp.arange(tf_pun[0], tf_pun[1]+savetimestep/2, savetimestep),
                              # time frame and time axis for saving the system's state
                              rtol, atol)  # relative and
ts = np.concatenate((ts, np.array(sol_punisher_active.ts)), axis=0)
xs = np.concatenate((xs, np.array(sol_punisher_active.ys)), axis=0)

# simulate toggle switch flipping by inducer pulse
orig_ptog11_acfrac = par_flip['p_tog11_ac_frac']  # save the original active fraction of p_tog11
orig_ptog21_acfrac = par_flip['p_tog21_ac_frac']  # save the original active fraction of p_tog21
par_flip['p_tog11_ac_frac'] = pulse_ptog11_acfrac  # set the active fraction of p_tog11 to the pulse value
par_flip['p_tog21_ac_frac'] = pulse_ptog21_acfrac  # set the active fraction of p_tog21 to the pulse value
x0_pulse = sol_punisher_active.ys[-1, :]  # simulation will resume from the last time point
sol_pulse = ode_sim(par_flip,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_pulse,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par_flip, circuit_genes), # synthetic gene parameters for calculating k values
                        tf_pulse, jnp.arange(tf_pulse[0], tf_pulse[1]+savetimestep/2, savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
flip_ts = np.concatenate((ts, np.array(sol_pulse.ts)), axis=0)
flip_xs = np.concatenate((xs, np.array(sol_pulse.ys)), axis=0)

# after the pulse, inducer levels return to the original state
par_flip['p_tog11_ac_frac'] = orig_ptog11_acfrac  # set the active fraction of p_tog11 to the pulse value
par_flip['p_tog21_ac_frac'] = orig_ptog21_acfrac  # set the active fraction of p_tog21 to the pulse value
x0_afterpulse = sol_pulse.ys[-1, :]  # simulation will resume from the last time point
sol_afterpulse = ode_sim(par_flip,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_afterpulse,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par_flip, circuit_genes),  # synthetic gene parameters for calculating k values
                        tf_afterpulse, jnp.arange(tf_afterpulse[0], tf_afterpulse[1], savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
flip_ts = np.concatenate((flip_ts, np.array(sol_afterpulse.ts)), axis=0)
flip_xs = np.concatenate((flip_xs, np.array(sol_afterpulse.ys)), axis=0)

# get growth rates
_, flip_ls, _, _, _, _, _, _ =cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(flip_ts, flip_xs, par_flip, circuit_genes, circuit_miscs, circuit_name2pos)

In [7]:
# PLOT HOW THE PUNISHER GETS SWITCHED ON BY A TRANSIENT FLUCTUATION IN BURDEN

# figs6a - TOGGLE PROTEIN LEVELS
flip_togprot_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        title="ds = " + np.format_float_positional(par['d_switch'], precision=3) + " 1/(nM*h); I = "+
            np.format_float_positional(par['p_switch_ac_frac'], precision=3),
        x_axis_label="Time since mutation, h",
        y_axis_label="Protein conc., nM",
        x_range=(-5,tf_afterpulse[1]-tf_pulse[0]),
        y_range=(0,7e4),
        tools="box_zoom,pan,hover,reset,save"
    )

# add shading to show when synthetic gene expression loss occurs
flip_togprot_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_pulse[1] - tf_pulse[0], tf_pulse[1] - tf_pulse[0]],
                            ys=[0, 2e5, 2e5, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
flip_togprot_figure.add_layout(bkmodels.Label(x=0, y=7e4,
                                x_offset=0, y_offset=-12,
                                text='Toggle 12 & 22 inducer pulse',
                                text_font_size='8pt',
                                text_align='left'))

# plot the toggle switch protein concentrations for tog11 and tog21
flip_togprot_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_tog11']], line_width=2, line_color=bkRGB(100, 149, 237),
           legend_label='tog11')
flip_togprot_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_tog21']], line_width=2, line_color=bkRGB(207, 181, 59),
           legend_label='tog21')

# plot the toggle switch protein concentrations for tog12 and tog22
flip_togprot_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_tog12']], line_width=2, line_color=bkRGB(127, 255, 212),
           legend_label='tog12')
flip_togprot_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_tog22']], line_width=2, line_color=bkRGB(252, 194, 0),
           legend_label='tog22')

# legend formatting
flip_togprot_figure.legend.location = "bottom_right"
flip_togprot_figure.legend.label_text_font_size = "8pt"
flip_togprot_figure.legend.padding = 3
flip_togprot_figure.legend.spacing = 2
flip_togprot_figure.legend.glyph_width = 10
flip_togprot_figure.legend.margin = 15


# set fonts
flip_togprot_figure.xaxis.axis_label_text_font_size = "8pt"
flip_togprot_figure.xaxis.major_label_text_font_size = "8pt"
flip_togprot_figure.yaxis.axis_label_text_font_size = "8pt"
flip_togprot_figure.yaxis.major_label_text_font_size = "8pt"
flip_togprot_figure.yaxis[0].formatter = bkmodels.PrintfTickFormatter(format="%.1e")

# FIG S6B: GROWTH RATES AND INTEGRASE LEVELS
flip_grint_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        title="ds = " + np.format_float_positional(par['d_switch'], precision=3) + " 1/(nM*h); I = "+
            np.format_float_positional(par['p_switch_ac_frac'], precision=3),
        x_axis_label="Time since mutation, h",
        y_axis_label="Cell growth rate, 1/h",
        x_range=(-5, tf_afterpulse[1] - tf_pulse[0]),
        y_range=(0.75, 1.6),
        tools="box_zoom,pan,hover,reset,save"
    )

flip_grint_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_pulse[1] - tf_pulse[0], tf_pulse[1] - tf_pulse[0]],
                            ys=[0, 1.75, 1.75, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
flip_grint_figure.add_layout(bkmodels.Label(x=0, y=1.6,
                                x_offset=0, y_offset=-12,
                                text='Toggle 12 & 22 inducer pulse',
                                text_font_size='8pt',
                                text_align='left'))
# plot the growth rate
flip_grint_figure.line(flip_ts - tf_pulse[0], flip_ls, line_width=2, line_color=bkRGB(0, 0, 0), legend_label='Growth rate')

# create an extra  y range for plotting integrase protein concentrations
flip_grint_figure.extra_y_ranges = {"p_int": bkmodels.Range1d(start=0, end=125)}
flip_grint_figure.add_layout(bkmodels.LinearAxis(y_range_name="p_int",
                                     axis_label="Integrase conc., nM",
                                     axis_line_color=bkRGB(255, 103, 0),
                                     major_tick_line_color=bkRGB(255, 103, 0),
                                     minor_tick_line_color=bkRGB(255, 103, 0)),
                 'right')  # add the alternative axis label to the figure

# plot the integrase protein concentrations
flip_grint_figure.line(flip_ts - tf_pulse[0], flip_xs[:, circuit_name2pos['p_int']], line_width=2, line_color=bkRGB(255, 103, 0),
           y_range_name="p_int", legend_label="Int. conc.")

# set fonts
flip_grint_figure.xaxis.axis_label_text_font_size = "8pt"
flip_grint_figure.xaxis.major_label_text_font_size = "8pt"
flip_grint_figure.yaxis.axis_label_text_font_size = "8pt"
flip_grint_figure.yaxis.major_label_text_font_size = "8pt"

# legend formatting
flip_grint_figure.legend.location = "top_right"
flip_grint_figure.legend.label_text_font_size = "8pt"
flip_grint_figure.legend.padding = 3
flip_grint_figure.legend.spacing = 2
flip_grint_figure.legend.glyph_width = 10
flip_grint_figure.legend.margin = 15

# show plots
flip_togprot_figure.output_backend = "svg"
flip_grint_figure.output_backend = "svg"
bkplot.show(bklayouts.column([flip_togprot_figure, flip_grint_figure]))

In [8]:
# SIMULATE HOW SLOWER ACTIVATION PREVENTS SWITCHING IN RESPONSE TO A TRANSIENT FLUCTUATION IN BURDEN

# smaller degradation rate (see Sharkov et al., 2001)
par_slowflip = par.copy()
par_slowflip['d_switch'] = 0.001944
par_slowflip['p_switch_ac_frac'] = par['p_switch_ac_frac'] / (1 + cellvars['chi_switch']) * (1 + (cellvars['chi_switch']/par['d_switch']) * par_slowflip['d_switch'])

# initial simulation to get the steady state without gene expression loss
ac_frac = par_slowflip['p_switch_ac_frac']
par_slowflip['p_switch_ac_frac'] = 0.0  # first with punisher inactive to avoid it being triggered
sol = ode_sim(par_slowflip,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par_slowflip, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf_prepun, jnp.arange(tf_prepun[0], tf_prepun[1]+savetimestep/2, savetimestep),  # time frame and time axis for saving the system's state
              rtol, atol)  # relative and
ts = np.array(sol.ts)
xs = np.array(sol.ys)
par_slowflip['p_switch_ac_frac'] = ac_frac
sol_punisher_active = ode_sim(par_slowflip,  # dictionary with model parameters
                              ode_with_circuit,  # ODE function for the cell with synthetic circuit
                              sol.ys[-1, :],
                              # initial condition VECTOR
                              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                              cellmodel_auxil.synth_gene_params_for_jax(par_slowflip, circuit_genes),
                              # synthetic gene parameters for calculating k values
                              tf_pun, jnp.arange(tf_pun[0], tf_pun[1]+savetimestep/2, savetimestep),
                              # time frame and time axis for saving the system's state
                              rtol, atol)  # relative and
ts = np.concatenate((ts, np.array(sol_punisher_active.ts)), axis=0)
xs = np.concatenate((xs, np.array(sol_punisher_active.ys)), axis=0)

# simulate toggle switch flipping by inducer pulse
orig_ptog11_acfrac = par_slowflip['p_tog11_ac_frac']  # save the original active fraction of p_tog11
orig_ptog21_acfrac = par_slowflip['p_tog21_ac_frac']  # save the original active fraction of p_tog21
par_slowflip['p_tog11_ac_frac'] = pulse_ptog11_acfrac  # set the active fraction of p_tog11 to the pulse value
par_slowflip['p_tog21_ac_frac'] = pulse_ptog21_acfrac  # set the active fraction of p_tog21 to the pulse value
x0_pulse = sol_punisher_active.ys[-1, :]  # simulation will resume from the last time point
sol_pulse = ode_sim(par_slowflip,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_pulse,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par_slowflip, circuit_genes), # synthetic gene parameters for calculating k values
                        tf_pulse, jnp.arange(tf_pulse[0], tf_pulse[1]+savetimestep/2, savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
slowflip_ts = np.concatenate((ts, np.array(sol_pulse.ts)), axis=0)
slowflip_xs = np.concatenate((xs, np.array(sol_pulse.ys)), axis=0)

# after the pulse, inducer levels return to the original state
par_slowflip['p_tog11_ac_frac'] = orig_ptog11_acfrac  # set the active fraction of p_tog11 to the pulse value
par_slowflip['p_tog21_ac_frac'] = orig_ptog21_acfrac  # set the active fraction of p_tog21 to the pulse value
x0_afterpulse = sol_pulse.ys[-1, :]  # simulation will resume from the last time point
sol_afterpulse = ode_sim(par_slowflip,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_afterpulse,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par_slowflip, circuit_genes),  # synthetic gene parameters for calculating k values
                        tf_afterpulse, jnp.arange(tf_afterpulse[0], tf_afterpulse[1], savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
slowflip_ts = np.concatenate((slowflip_ts, np.array(sol_afterpulse.ts)), axis=0)
slowflip_xs = np.concatenate((slowflip_xs, np.array(sol_afterpulse.ys)), axis=0)

# get growth rates
_, slowflip_ls, _, _, _, _, _, _ =cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(slowflip_ts, slowflip_xs, par_slowflip, circuit_genes, circuit_miscs, circuit_name2pos)

In [9]:
# PLOT HOW SLOWER ACTIVATION PREVENTS SWITCHING IN RESPONSE TO A TRANSIENT FLUCTUATION IN BURDEN

# FIG S6C - TOGGLE PROTEIN LEVELS
slowflip_togprot_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        title="ds = " + np.format_float_positional(par_slowflip['d_switch'], precision=3) + " 1/(nM*h); I = "+
            np.format_float_positional(par_slowflip['p_switch_ac_frac'], precision=3),
        x_axis_label="Time since mutation, h",
        y_axis_label="Protein conc., nM",
        x_range=(-5,tf_afterpulse[1]-tf_pulse[0]),
        y_range=(0,7e4),
        tools="box_zoom,pan,hover,reset,save"
    )

# add shading to show when synthetic gene expression loss occurs
slowflip_togprot_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_pulse[1] - tf_pulse[0], tf_pulse[1] - tf_pulse[0]],
                            ys=[0, 2e5, 2e5, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
slowflip_togprot_figure.add_layout(bkmodels.Label(x=0, y=7e4,
                                x_offset=0, y_offset=-12,
                                text='Toggle 12 & 22 inducer pulse',
                                text_font_size='8pt',
                                text_align='left'))

# plot the toggle switch protein concentrations for tog11 and tog21
slowflip_togprot_figure.line(slowflip_ts - tf_pulse[0], slowflip_xs[:, circuit_name2pos['p_tog11']], line_width=2, line_color=bkRGB(100, 149, 237),
           legend_label='tog11')
slowflip_togprot_figure.line(slowflip_ts - tf_pulse[0], slowflip_xs[:, circuit_name2pos['p_tog21']], line_width=2, line_color=bkRGB(207, 181, 59),
           legend_label='tog21')

# plot the toggle switch protein concentrations for tog12 and tog22
slowflip_togprot_figure.line(slowflip_ts - tf_pulse[0], slowflip_xs[:, circuit_name2pos['p_tog12']], line_width=2, line_color=bkRGB(127, 255, 212),
           legend_label='tog12')
slowflip_togprot_figure.line(slowflip_ts - tf_pulse[0], slowflip_xs[:, circuit_name2pos['p_tog22']], line_width=2, line_color=bkRGB(252, 194, 0),
           legend_label='tog22')

# legend formatting
slowflip_togprot_figure.legend.location = "bottom_right"
slowflip_togprot_figure.legend.label_text_font_size = "8pt"
slowflip_togprot_figure.legend.padding = 3
slowflip_togprot_figure.legend.spacing = 2
slowflip_togprot_figure.legend.glyph_width = 10
slowflip_togprot_figure.legend.margin = 15


# set fonts
slowflip_togprot_figure.xaxis.axis_label_text_font_size = "8pt"
slowflip_togprot_figure.xaxis.major_label_text_font_size = "8pt"
slowflip_togprot_figure.yaxis.axis_label_text_font_size = "8pt"
slowflip_togprot_figure.yaxis.major_label_text_font_size = "8pt"
slowflip_togprot_figure.yaxis[0].formatter = bkmodels.PrintfTickFormatter(format="%.1e")

# FIG S6D: GROWTH RATES AND INTEGRASE LEVELS
slowflip_grint_figure = bkplot.figure(
        frame_width=200,
        frame_height=125,
        title="ds = " + np.format_float_positional(par_slowflip['d_switch'], precision=3) + " 1/(nM*h); I = "+
            np.format_float_positional(par_slowflip['p_switch_ac_frac'], precision=3),
        x_axis_label="Time since mutation, h",
        y_axis_label="Cell growth rate, 1/h",
        x_range=(-5, tf_afterpulse[1] - tf_pulse[0]),
        y_range=(0.75, 1.6),
        tools="box_zoom,pan,hover,reset,save"
    )

slowflip_grint_figure.add_layout(
    bkmodels.PolyAnnotation(xs=[0, 0, tf_pulse[1] - tf_pulse[0], tf_pulse[1] - tf_pulse[0]],
                            ys=[0, 1.75, 1.75, 0],
                            line_width=0, line_alpha=0,
                            fill_color=bkRGB(100, 100, 100, 0.25)))
slowflip_grint_figure.add_layout(bkmodels.Label(x=0, y=1.6,
                                x_offset=0, y_offset=-12,
                                text='Toggle 12 & 22 inducer pulse',
                                text_font_size='8pt',
                                text_align='left'))
# plot the growth rate
slowflip_grint_figure.line(slowflip_ts - tf_pulse[0], slowflip_ls, line_width=2, line_color=bkRGB(0, 0, 0), legend_label='Growth rate')

# create an extra  y range for plotting integrase protein concentrations
slowflip_grint_figure.extra_y_ranges = {"p_int": bkmodels.Range1d(start=0, end=125)}
slowflip_grint_figure.add_layout(bkmodels.LinearAxis(y_range_name="p_int",
                                     axis_label="Integrase conc., nM",
                                     axis_line_color=bkRGB(255, 103, 0),
                                     major_tick_line_color=bkRGB(255, 103, 0),
                                     minor_tick_line_color=bkRGB(255, 103, 0)),
                 'right')  # add the alternative axis label to the figure

# plot the integrase protein concentrations
slowflip_grint_figure.line(slowflip_ts - tf_pulse[0], slowflip_xs[:, circuit_name2pos['p_int']], line_width=2, line_color=bkRGB(255, 103, 0),
           y_range_name="p_int", legend_label="Int. conc.")

# set fonts
slowflip_grint_figure.xaxis.axis_label_text_font_size = "8pt"
slowflip_grint_figure.xaxis.major_label_text_font_size = "8pt"
slowflip_grint_figure.yaxis.axis_label_text_font_size = "8pt"
slowflip_grint_figure.yaxis.major_label_text_font_size = "8pt"

# legend formatting
slowflip_grint_figure.legend.location = "right"
slowflip_grint_figure.legend.label_text_font_size = "8pt"
slowflip_grint_figure.legend.padding = 3
slowflip_grint_figure.legend.spacing = 2
slowflip_grint_figure.legend.glyph_width = 10
slowflip_grint_figure.legend.margin = 15

# show plots
slowflip_togprot_figure.output_backend = "svg"
slowflip_grint_figure.output_backend = "svg"
bkplot.show(bklayouts.column([slowflip_togprot_figure, slowflip_grint_figure]))

In [10]:
# DEFINE THE RANGES OF SHARE OF SWITCH PROTEINS BOUND BY THE INDUCER AND PROTEASE INDUCTION VALUES

# bound (active) fraction range
ac_frac_range=np.linspace(0,1,80)

# protease induction range
dswitch_range=np.linspace(0,0.02,80)
dswitch_default = par['d_switch'] # back up the default switch protein degradation rate

In [11]:
# ACTIVE FRACTION VS PROTEASE INDUCTION: FIND SWITCHING THRESHOLDS AND GUARANTEED FOLD CHANGES

# get a mesh grid, then flatten its x and y coordinates into a single linear array
ac_frac_mesh, dswitch_mesh = np.meshgrid(ac_frac_range, dswitch_range)
ac_frac_mesh_ravel = ac_frac_mesh.ravel()
dswitch_mesh_ravel = dswitch_mesh.ravel()

# specify vmapping axes
par_vmapping_axes = {}
for key in par.keys():
    if (key == 'p_switch_ac_frac' or key=='d_switch'):
        par_vmapping_axes[key] = 0
    else:
        par_vmapping_axes[key] = None
cellvars_vmapping_axes = {}
for key in cellvars.keys():
    if(key== 'chi_switch'):
        cellvars_vmapping_axes[key] = 0
    else:
        cellvars_vmapping_axes[key] = None
        
# make a vmappable parameter dictionary
par_for_existence = par.copy()
par_for_existence['p_switch_ac_frac'] = jnp.array(ac_frac_mesh_ravel)
par_for_existence['d_switch'] = jnp.array(dswitch_mesh_ravel)
# make a vmappable cellular variable dictionary
cellvars_for_existence = cellvars.copy()
cellvars_for_existence['chi_switch'] = (cellvars['chi_switch']/dswitch_default) * jnp.array(dswitch_mesh_ravel)

# # make the checking function vmappable
vmapped_check_if_threshold_exists = jax.jit(jax.vmap(check_if_threshold_exists,
                                                     in_axes=(par_vmapping_axes, cellvars_vmapping_axes)))

# find for which parameter combinations the switching threshold exists
threshold_exists = np.array(vmapped_check_if_threshold_exists(par_for_existence, cellvars_for_existence))

# from now on, only consider parameter combinations where the threshold bifurcation point exists
indices_where_threshold_exists = []
for i in range(0, len(threshold_exists)):
    if (threshold_exists[i]):
        indices_where_threshold_exists.append(i)
ac_frac_mesh_ravel_exists = ac_frac_mesh_ravel[indices_where_threshold_exists]
dswitch_mesh_ravel_exists = dswitch_mesh_ravel[indices_where_threshold_exists]

# make a vmappable parameter dictionary
par_for_threshold_gfchanges = par.copy()
par_for_threshold_gfchanges['p_switch_ac_frac'] = jnp.array(ac_frac_mesh_ravel_exists)
par_for_threshold_gfchanges['d_switch'] = jnp.array(dswitch_mesh_ravel_exists)
# make a vmappable cellular variable dictionary
cellvars_for_threshold_gfchanges = cellvars.copy()
cellvars_for_threshold_gfchanges['chi_switch'] = (cellvars['chi_switch']/dswitch_default) * jnp.array(dswitch_mesh_ravel_exists)

# make the threshold and guaranteed fold change retrieval function vmappable
vmapped_threshold_gfchanges = jax.jit(jax.vmap(threshold_gfchanges,
                                               in_axes=(par_vmapping_axes, cellvars_vmapping_axes)))

# find switching thresholds and guaranteed fold changes for the parameter combinations where the switching threshold exists
thresholds_gfchanges = np.array(vmapped_threshold_gfchanges(par_for_threshold_gfchanges, cellvars_for_threshold_gfchanges))

# get the switching thresholds and guaranteed fold changes
xi_thresholds = thresholds_gfchanges[:, 1]
gfchange_intact = thresholds_gfchanges[:, 4]

In [12]:
# ACTIVE FRACTION VS PROTEASE INDUCTION: FIND BURDEN CONTOURS

# fill the points where no threshold exists with INFS
xi_thresholds_for_contour_ravel = np.zeros(threshold_exists.shape)  # initialise
last_index_in_exist_list = 0
for i in range(0, len(xi_thresholds_for_contour_ravel)):
    if (i == indices_where_threshold_exists[last_index_in_exist_list]):
        xi_thresholds_for_contour_ravel[i] = xi_thresholds[last_index_in_exist_list]
        if (last_index_in_exist_list < len(indices_where_threshold_exists) - 1):
            last_index_in_exist_list += 1
    else:
        xi_thresholds_for_contour_ravel[i] = np.inf
xi_thresholds_for_contour = xi_thresholds_for_contour_ravel.reshape(len(dswitch_range),
                                                                    len(ac_frac_range)).T

# create a contour generator
threshold_cgen = cgen(x=dswitch_range, y=ac_frac_range,
                      z=xi_thresholds_for_contour)

# contours to be found: 1) all synth. genes functional; 2) just the CAT and protease genes functional
xi_one_toggle = cellvars['xi_a'] + cellvars['xi_r'] + cellvars['xi_cat'] + cellvars['xi_prot'] + cellvars['xi_other_genes']/2
xi_with_all_genes = xi_one_toggle + cellvars['xi_other_genes']/2
xi_contours = {'values': [xi_with_all_genes, xi_one_toggle],
               'legends': ['All toggles\nfunctional', 'One\ntoggle mutated'],
               'dashes': ['dashed','solid']}
# find burden contour lines
xi_contours['contour lines'] = []
for i in range(0, len(xi_contours['values'])):
    xi_contours['contour lines'].append(threshold_cgen.lines(xi_contours['values'][i]))

In [13]:
# ACTIVE FRACTIONS VS PROTEASE INDUCTION: FIND COMBINATIONS STANDING FOR THE SAME SWITCHING THRESHOLD

ac_fracs_same_threshold = par['p_switch_ac_frac'] / (1 + cellvars['chi_switch']) * (1 + (cellvars['chi_switch']/dswitch_default) * dswitch_range)

In [14]:
# ACTIVE FRACTIONS VS PROTEASE INDUCTION: PLOT

# rect widths and heights
rect_widths_along_x_axis = np.zeros(len(dswitch_range))
rect_widths_along_x_axis[0] = dswitch_range[1] - dswitch_range[0]
for i in range(1, len(dswitch_range)):
    rect_widths_along_x_axis[i] = ((dswitch_range[i] - dswitch_range[i - 1]) -
                                   rect_widths_along_x_axis[i - 1] / 2) * 2
rect_heights_along_y_axis = np.zeros(len(ac_frac_range))
rect_heights_along_y_axis[0] = ac_frac_range[1] - ac_frac_range[0]
for i in range(1, len(ac_frac_range)):
    rect_heights_along_y_axis[i] = ((ac_frac_range[i] - ac_frac_range[i - 1]) - rect_heights_along_y_axis[i - 1] / 2) * 2
rect_widths_ravel_exists = np.zeros(dswitch_mesh_ravel_exists.shape)
rect_heights_ravel_exists = np.zeros(ac_frac_mesh_ravel_exists.shape)
for i in range(0, len(dswitch_mesh_ravel_exists)):
    fprot_where = np.argwhere(
        dswitch_range == dswitch_mesh_ravel_exists[i])  # locate the baseline value in the protease gene induction range
    rect_widths_ravel_exists[i] = rect_widths_along_x_axis[fprot_where[0][0]]*1.25
    ac_frac_where = np.argwhere(ac_frac_range == ac_frac_mesh_ravel_exists[i])  # locate the eta value in the active fractions range
    rect_heights_ravel_exists[i] = rect_heights_along_y_axis[ac_frac_where[0][0]]*1.25
    
# make a dataframe for the heatmap of guaranteed fold changes
heatmap_df = pd.DataFrame({'d_switch': dswitch_mesh_ravel_exists, 'I': ac_frac_mesh_ravel_exists,
                           'gfchange_intact': gfchange_intact,
                           'rect_width': rect_widths_ravel_exists, 'rect_height': rect_heights_ravel_exists})

# plot FigS6G
gfchange_intact_figure = bkplot.figure(
    frame_width=200,
    frame_height=200,
    x_axis_label="ds (switch prot. degradation rate), 1/(nM*h)",
    y_axis_label="I (share of inducer-bound\nswitch prots.)",
    x_range=(min(dswitch_range), max(dswitch_range)),
    y_range=(0.1, max(ac_frac_range)),
    tools='pan,box_zoom,reset,save'
)
gfchange_intact_figure.output_backend = "svg"

# plot the heatmap
rects = gfchange_intact_figure.rect(x="d_switch", y="I", source=heatmap_df,
                                 width='rect_width', height='rect_height',
                                 fill_color=bktransform.log_cmap('gfchange_intact',
                                                                    bkpalettes.Plasma256,
                                                                    low=1,
                                                                    high=max(heatmap_df['gfchange_intact'])),
                                 line_width=0,line_alpha=0)
# add colour bar
gfchange_intact_figure.add_layout(rects.construct_color_bar(
    major_label_text_font_size="8pt",
    ticker=bkmodels.FixedTicker(ticks=[1, 10, 100, 1000, 10000]),
    formatter=bkmodels.PrintfTickFormatter(format="%e"),
    label_standoff=6,
    border_line_color=None,
    padding=5
), 'right')

# plot the points where the same switching threshold is reached
gfchange_intact_figure.line(x=dswitch_range, y=ac_fracs_same_threshold,
            line_width=2, line_color='white', line_dash='solid')

# plot the burden contours
for i in range(0,len(xi_contours['values'])):
    for j in range(0,len(xi_contours['contour lines'][i])):
        gfchange_intact_figure.line(xi_contours['contour lines'][i][j][:, 0], xi_contours['contour lines'][i][j][:, 1],
                    line_dash=xi_contours['dashes'][i],
                    #legend_label=xi_contours['legends'][i],
                    line_width=2, line_color='black')

# add burden contour labels
gfchange_intact_figure.add_layout(bkmodels.Label(x=xi_contours['contour lines'][0][0][0, 0],
                                 y=xi_contours['contour lines'][0][0][0, 1],
                                 x_offset=4, y_offset=48,
                                 text_align='left',
                                 text=xi_contours['legends'][0],
                                 text_font_size='8pt',
                                 text_color='black'))
gfchange_intact_figure.add_layout(bkmodels.Label(x=xi_contours['contour lines'][1][0][0, 0],
                                 y=xi_contours['contour lines'][1][0][0, 1],
                                 x_offset=100, y_offset=0,
                                 text_align='right',
                                 text=xi_contours['legends'][1],
                                 text_font_size='8pt',
                                 text_color='black'))

# mark the point with default parameters
gfchange_intact_figure.scatter(marker='x', x=dswitch_default, y=[par['p_switch_ac_frac']],
                               size=10, color=bkRGB(72, 209, 204),line_width=4)

# mark the point with slowed-down switching
gfchange_intact_figure.scatter(marker='x', x=par_slowflip['d_switch'], y=[par_slowflip['p_switch_ac_frac']],
                                 size=10, color=bkRGB(0, 0, 0),line_width=4)

# font size
gfchange_intact_figure.xaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure.xaxis.major_label_text_font_size = "8pt"
gfchange_intact_figure.yaxis.axis_label_text_font_size = "8pt"
gfchange_intact_figure.yaxis.major_label_text_font_size = "8pt"

# show plot
bkplot.show(gfchange_intact_figure)

# save plot
bkplot.output_file("FigS6G.html")
bkplot.save(gfchange_intact_figure)

'/mnt/c/Users/ersat/CODE/punisher/FigS6/FigS6G.html'

In [15]:
# # GET SWITCHING TIMESCALES FOR THE SAME SWITCHING THRESHOLD AND DIFFERENT PROTEASE INDUCTION VALUES
# 
# # set deterministic simulation parameters
# savetimestep_preloss = 1  # save time step: before timescale evaluation
# savetimestep_afterloss = 0.01  # save time step: after timescale evaluation
# rtol = 1e-6  # relative tolerance for the ODE solver
# atol = 1e-6  # absolute tolerance for the ODE solver
# tf_nopun = (0,25) # time frame for simulation before the punisher comes online
# tf_pun = (25,50) # time frame for simulation after the punisher comes online
# tf_afterloss = (50,100) # time frame for simulation after burdensome gene loss
# 
# # initialise the array of switching times
# dswitch_range_times=np.linspace(dswitch_range[0],0.02,25)
# ac_fracs_same_threshold_times=par['p_switch_ac_frac'] / (1 + cellvars['chi_switch']) * (1 + (cellvars['chi_switch']/dswitch_default) * dswitch_range_times)
# cross_times = np.zeros_like(dswitch_range_times)
# 
# # cycle through all parameter combinations
# for comb_cntr in range(0, len(dswitch_range_times)):
#     print('Simulating for d_switch = ', dswitch_range_times[comb_cntr])
# 
#     # copy system parameters from the default set
#     par_timescales = par.copy()
#     par_timescales['k_sxf'] = 0.0  # ignore integrase action
#     cellvars_timescales = cellvars.copy()
#     # set the parameters of the system to the combination considered
#     par_timescales['p_switch_ac_frac'] = ac_fracs_same_threshold_times[comb_cntr]
#     par_timescales['d_switch'] = dswitch_range_times[comb_cntr]
#     cellvars_timescales['chi_switch'] = (cellvars['chi_switch']/dswitch_default) * par_timescales['d_switch']
# 
#     # get the border between the equilibria's basins of attraction
#     border = find_basin_border(par_timescales, cellvars_timescales)
# 
#     # simulate the loss of the burdensome gene
#     # initial simulation to get the steady state without gene expression loss
#     p_switch_ac_frac = par_timescales['p_switch_ac_frac']
#     par_timescales['p_switch_ac_frac'] = 0.0  # set the burdensome gene expression to present
#     sol=ode_sim(par_timescales,    # dictionary with model parameters
#                 ode_with_circuit,   #  ODE function for the cell with synthetic circuit
#                 cellmodel_auxil.x0_from_init_conds(init_conds,circuit_genes,circuit_miscs),  # initial condition VECTOR
#                 len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
#                 cellmodel_auxil.synth_gene_params_for_jax(par_timescales,circuit_genes), # synthetic gene parameters for calculating k values
#                 tf_nopun, jnp.arange(tf_nopun[0], tf_nopun[1], savetimestep_preloss), # time frame and time axis for saving the system's state
#                 rtol, atol, # relative and absolute tolerances
#                 solver=Heun()   # Heun solver for faster simulations
#                 )    
#     ts_nopun=np.array(sol.ts)
#     xs_nopun=np.array(sol.ys)
#     par_timescales['p_switch_ac_frac'] = p_switch_ac_frac  # set the burdensome gene expression back to present
#     sol=ode_sim(par_timescales,    # dictionary with model parameters
#                 ode_with_circuit,   #  ODE function for the cell with synthetic circuit
#                 xs_nopun[-1,:],  # initial condition VECTOR
#                 len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
#                 cellmodel_auxil.synth_gene_params_for_jax(par_timescales,circuit_genes), # synthetic gene parameters for calculating k values
#                 tf_pun, jnp.arange(tf_pun[0], tf_pun[1], savetimestep_preloss), # time frame and time axis for saving the system's state
#                 rtol, atol, # relative and absolute tolerances
#                 solver=Heun()   # Heun solver for faster simulations
#                 )    
#     ts_preloss=np.array(sol.ts)
#     xs_preloss=np.array(sol.ys)
# 
#     # simulating mutation of one toggle switch
#     x0_afterloss=sol.ys[-1,:]  # simulation will resume from the last time point
#     par_timescales['func_tog11'] = 0.0  # set the burdensome gene expression to zero
#     par_timescales['func_tog12'] = 0.0  # set the burdensome gene expression to zero
#     sol=ode_sim(par_timescales,    # dictionary with model parameters
#                 ode_with_circuit,   #  ODE function for the cell with synthetic circuit
#                 x0_afterloss,  # initial condition VECTOR
#                 len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
#                 cellmodel_auxil.synth_gene_params_for_jax(par_timescales,circuit_genes), # synthetic gene parameters for calculating k values
#                 tf_afterloss, jnp.arange(tf_afterloss[0], tf_afterloss[1], savetimestep_afterloss), # time frame and time axis for saving the system's state
#                 rtol, atol, # relative and absolute tolerances
#                 solver=Heun()   # Heun solver for faster simulations
#                 )    
#     ts_afterloss = np.array(sol.ts)
#     xs_afterloss = np.array(sol.ys)
# 
#     # get the first time point where p_switch crosses the border
#     cross_times[comb_cntr] = ts_afterloss[np.argwhere(xs_afterloss[:,circuit_name2pos['p_switch']] > border)[0][0]]-ts_afterloss[0]

In [16]:
# # PLOT THE SWITCHING TIMESCALES
# 
# timescales_figure = bkplot.figure(
#     frame_width=200,
#     frame_height=200,
#     x_axis_label="ds (switch prot. degradation rate), 1/(nM*h)",
#     y_axis_label="Time to irreversible\nswitching, h",
#     x_range=(min(dswitch_range_times), max(dswitch_range_times)),
#     tools='pan,box_zoom,reset,save'
# )
# timescales_figure.output_backend = 'svg'
# 
# # plot the switching timescales
# timescales_figure.line(x=dswitch_range_times, y=cross_times,
#             line_width=2, line_color=bkRGB(222, 49, 99))
# 
# # mark the point with default parameters
# cross_time_default = np.interp(dswitch_default, dswitch_range_times, cross_times)
# timescales_figure.scatter(marker='x', x=dswitch_default, y=cross_time_default,
#                                size=10, color=bkRGB(72, 209, 204),line_width=4)
# 
# # mark the point with slowed-down switching
# cross_time_slowflip = np.interp(par_slowflip['d_switch'], dswitch_range_times, cross_times)
# timescales_figure.scatter(marker='x', x=par_slowflip['d_switch'], y=cross_time_slowflip,
#                                size=10, color='black',line_width=4)
# 
# # font size
# timescales_figure.xaxis.axis_label_text_font_size = "8pt"
# timescales_figure.xaxis.major_label_text_font_size = "8pt"
# timescales_figure.yaxis.axis_label_text_font_size = "8pt"
# timescales_figure.yaxis.major_label_text_font_size = "8pt"
# 
# # show plot
# bkplot.show(timescales_figure)                        

In [17]:
# ALTERNATIVE: ADJUST THE SWITCHING THRESHOLD

ac_frac_default = par['p_switch_ac_frac'] # save the default active fraction

# set the range of I values that produce appropriate switching thresholds
ac_frac_range_times=np.linspace(0.863,0.883,21)

print('I values considered: ', ac_frac_range_times)

I values considered:  [0.863 0.864 0.865 0.866 0.867 0.868 0.869 0.87  0.871 0.872 0.873 0.874
 0.875 0.876 0.877 0.878 0.879 0.88  0.881 0.882 0.883]


In [18]:
# set deterministic simulation parameters
savetimestep_preloss = 1  # save time step: before timescale evaluation
savetimestep_afterloss = 0.01  # save time step: after timescale evaluation
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver
tf_nopun = (0,25) # time frame for simulation before the punisher comes online
tf_pun = (25,50) # time frame for simulation after the punisher comes online
tf_afterloss = (50,100) # time frame for simulation after burdensome gene loss

# initialise the array of switching times
cross_times_for_ac_frac=np.zeros_like(ac_frac_range_times)

# cycle through all active fractions
for comb_cntr in range(0, len(ac_frac_range_times)):
    print('Simulating for I = ', ac_frac_range_times[comb_cntr])

    # copy system parameters from the default set
    par_timescales = par.copy()
    par_timescales['k_sxf'] = 0.0  # ignore integrase action
    cellvars_timescales = cellvars.copy()
    # set the parameters of the system to the combination considered
    par_timescales['p_switch_ac_frac'] = ac_frac_range_times[comb_cntr]

    # get the border between the equilibria's basins of attraction
    border = find_basin_border(par_timescales, cellvars_timescales)

    # simulate the loss of the burdensome gene
    # initial simulation to get the steady state without gene expression loss
    p_switch_ac_frac = par_timescales['p_switch_ac_frac']
    par_timescales['p_switch_ac_frac'] = 0.0  # set the burdensome gene expression to present
    sol=ode_sim(par_timescales,    # dictionary with model parameters
                ode_with_circuit,   #  ODE function for the cell with synthetic circuit
                cellmodel_auxil.x0_from_init_conds(init_conds,circuit_genes,circuit_miscs),  # initial condition VECTOR
                len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                cellmodel_auxil.synth_gene_params_for_jax(par_timescales,circuit_genes), # synthetic gene parameters for calculating k values
                tf_nopun, jnp.arange(tf_nopun[0], tf_nopun[1], savetimestep_preloss), # time frame and time axis for saving the system's state
                rtol, atol, # relative and absolute tolerances
                solver=Heun()   # Heun solver for faster simulations
                )    
    ts_nopun=np.array(sol.ts)
    xs_nopun=np.array(sol.ys)
    par_timescales['p_switch_ac_frac'] = p_switch_ac_frac  # set the burdensome gene expression back to present
    sol=ode_sim(par_timescales,    # dictionary with model parameters
                ode_with_circuit,   #  ODE function for the cell with synthetic circuit
                xs_nopun[-1,:],  # initial condition VECTOR
                len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                cellmodel_auxil.synth_gene_params_for_jax(par_timescales,circuit_genes), # synthetic gene parameters for calculating k values
                tf_pun, jnp.arange(tf_pun[0], tf_pun[1], savetimestep_preloss), # time frame and time axis for saving the system's state
                rtol, atol, # relative and absolute tolerances
                solver=Heun()   # Heun solver for faster simulations
                )    
    ts_preloss=np.array(sol.ts)
    xs_preloss=np.array(sol.ys)

    # simulating mutation of one toggle switch
    x0_afterloss=sol.ys[-1,:]  # simulation will resume from the last time point
    par_timescales['func_tog11'] = 0.0  # set the burdensome gene expression to zero
    par_timescales['func_tog12'] = 0.0  # set the burdensome gene expression to zero
    sol=ode_sim(par_timescales,    # dictionary with model parameters
                ode_with_circuit,   #  ODE function for the cell with synthetic circuit
                x0_afterloss,  # initial condition VECTOR
                len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                cellmodel_auxil.synth_gene_params_for_jax(par_timescales,circuit_genes), # synthetic gene parameters for calculating k values
                tf_afterloss, jnp.arange(tf_afterloss[0], tf_afterloss[1], savetimestep_afterloss), # time frame and time axis for saving the system's state
                rtol, atol, # relative and absolute tolerances
                solver=Heun()   # Heun solver for faster simulations
                )    
    ts_afterloss = np.array(sol.ts)
    xs_afterloss = np.array(sol.ys)

    # get the first time point where p_switch crosses the border
    cross_times_for_ac_frac[comb_cntr] = ts_afterloss[np.argwhere(xs_afterloss[:,circuit_name2pos['p_switch']] > border)[0][0]]-ts_afterloss[0]

Simulating for I =  0.863
Simulating for I =  0.864
Simulating for I =  0.865
Simulating for I =  0.866
Simulating for I =  0.867
Simulating for I =  0.868
Simulating for I =  0.869
Simulating for I =  0.87
Simulating for I =  0.871
Simulating for I =  0.872
Simulating for I =  0.873
Simulating for I =  0.874
Simulating for I =  0.875
Simulating for I =  0.876
Simulating for I =  0.877
Simulating for I =  0.878
Simulating for I =  0.879
Simulating for I =  0.88
Simulating for I =  0.881
Simulating for I =  0.882
Simulating for I =  0.883


In [19]:
# PLOT THE SWITCHING TIMESCALES
timescales_for_ac_frac_figure = bkplot.figure(
    frame_width=200,
    frame_height=125,
    x_axis_label="I (share of inducer-bound switch proteins)",
    y_axis_label="Time to irreversible\nswitching, h",
    x_range=(min(ac_frac_range_times), max(ac_frac_range_times)),
    tools='pan,box_zoom,reset,save'
)
timescales_for_ac_frac_figure.output_backend = 'svg'

# plot the switching timescales
timescales_for_ac_frac_figure.line(x=ac_frac_range_times, y=cross_times_for_ac_frac,
            line_width=2, line_color=bkRGB(222, 49, 99))

# mark the point with default parameters
cross_time_for_ac_frac_default = np.interp(ac_frac_default, ac_frac_range_times, cross_times_for_ac_frac)
timescales_for_ac_frac_figure.scatter(marker='x', x=ac_frac_default, y=cross_time_for_ac_frac_default,
                               size=10, color=bkRGB(72, 209, 204),line_width=4)
# font size
timescales_for_ac_frac_figure.xaxis.axis_label_text_font_size = "8pt"
timescales_for_ac_frac_figure.xaxis.major_label_text_font_size = "8pt"
timescales_for_ac_frac_figure.yaxis.axis_label_text_font_size = "8pt"
timescales_for_ac_frac_figure.yaxis.major_label_text_font_size = "8pt"

# show plot
bkplot.show(timescales_for_ac_frac_figure)

In [20]:
# ALTERNATIVE: ADJUST THE SWITCHING THRESHOLD
# PLOT BIFURCATION TRESHOLD AS A FUNCTION OF CHEMICAL INDUCTION AT GIVEN SWITCH PROTEIN DEGRADATION RATE

# specify vmapping axes
par_vmapping_axes_vs_ac_frac = {}
for key in par.keys():
    if (key == 'p_switch_ac_frac'):
        par_vmapping_axes_vs_ac_frac[key] = 0
    else:
        par_vmapping_axes_vs_ac_frac[key] = None
cellvars_vmapping_axes_vs_ac_frac = {}
for key in cellvars.keys():
    cellvars_vmapping_axes_vs_ac_frac[key] = None
    
# make a vmappable parameter dictionary
par_for_threshold_gfchanges_vs_ac_frac = par.copy()
par_for_threshold_gfchanges_vs_ac_frac['p_switch_ac_frac'] = ac_frac_range_times
# make the threshold and guaranteed fold change retrieval function vmappable
vmapped_threshold_gfchanges_vs_ac_frac = jax.jit(jax.vmap(threshold_gfchanges,
                                               in_axes=(par_vmapping_axes_vs_ac_frac, cellvars_vmapping_axes_vs_ac_frac)))

# find switching thresholds and guaranteed fold changes for the parameter combinations where the switching threshold exists
thresholds_gfchanges_vs_ac_frac = np.array(vmapped_threshold_gfchanges_vs_ac_frac(par_for_threshold_gfchanges_vs_ac_frac, cellvars))

# get the switching thresholds and guaranteed fold changes
xi_thresholds = thresholds_gfchanges_vs_ac_frac[:, 1]
gfchange_intact = thresholds_gfchanges_vs_ac_frac[:, 4]

In [21]:
# PLOT THE SWITCHING THRESHOLDS
thresholds_for_ac_frac_figure = bkplot.figure(
    frame_width=200,
    frame_height=125,
    x_axis_label="I (share of inducer-bound switch proteins)",
    y_axis_label="Ξ (switching\nthreshold burden)",
    x_range=(min(ac_frac_range_times), max(ac_frac_range_times)),
    tools='pan,box_zoom,reset,save'
)
thresholds_for_ac_frac_figure.output_backend = 'svg'

# plot the switching thresholds
thresholds_for_ac_frac_figure.line(x=ac_frac_range_times, y=xi_thresholds,
            line_width=2, line_color=bkRGB(222, 49, 99))

# mark the point with default parameters
xi_threshold_default = np.interp(ac_frac_default, ac_frac_range_times, xi_thresholds)
thresholds_for_ac_frac_figure.scatter(marker='x', x=ac_frac_default, y=xi_threshold_default,
                               size=10, color=bkRGB(72, 209, 204),line_width=4)

# font size
thresholds_for_ac_frac_figure.xaxis.axis_label_text_font_size = "8pt"
thresholds_for_ac_frac_figure.xaxis.major_label_text_font_size = "8pt"
thresholds_for_ac_frac_figure.yaxis.axis_label_text_font_size = "8pt"
thresholds_for_ac_frac_figure.yaxis.major_label_text_font_size = "8pt"

# show plot
bkplot.show(thresholds_for_ac_frac_figure)